In [1]:
!pip install transformers datasets sentencepiece

## 파이프라인

In [2]:
from transformers import pipeline
import pandas as pd

text = """안녕하세요, 지난 주 독일 아마존 온라인 스토어에서 옵티머스프라임 액션 피규어를 주문했는데,
받아보니 메가트론 피규어가 와 있더라구요. 디셉티콘은 평생 적인데, 이런 실수는 좀 곤란하네요.
메가트론 대신 주문한 옵티머스프라임 피규어로 교환해주시면 감사하겠습니다.
구매 내역 사본도 함께 보냈습니다. 빠른 답변 부탁드릴게요."""

text-classification

In [25]:
classifier = pipeline("text-classification", model = "WhitePeak/bert-base-cased-Korean-sentiment")   #LABEL_0: negative LABEL_1: positive

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,LABEL_0,0.974917


ner

In [26]:
ner_tagger = pipeline("ner", model = "Leo97/KoELECTRA-small-v3-modu-ner",
                      aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,DT,0.955350,지난 주,7,11
1,LC,0.971672,독일,12,14
2,AF,0.343778,아마존,15,18
3,AF,0.423908,옵티머스,29,33
4,TM,0.371845,##프라임,33,36


question answering

In [28]:
reader = pipeline("question-answering", model = "yjgwak/klue-bert-base-finetuned-squad-kor-v1")
question = "고객이 원하는 것은 뭐야?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

,score,start,end,answer
0,0.032955,116,139,주문한 옵티머스프라임 피규어로 교환해주시면


summarize

In [33]:
summarizer = pipeline("summarization", model = "eenzeenee/t5-base-korean-summarization")
outputs = summarizer(text, max_length = 100, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

옵티머스프라임 액션 피규어를 주문했는데 메가트론 피규어가 와 있어서 디셉티콘은 평생 적인데 이런 실수는 곤란하다. 메가트론 대신 주문한 옵티머스프라임 피규어로 교환해 주면 감사하겠다.


generate

In [9]:
generator = pipeline("text-generation", model = "skt/kogpt2-base-v2")
# del generator

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [10]:
response = "아이고 손님 죄송합니다. 당장"
prompt = text + "\n\n고객센터 답변:\n" + response
outputs = generator(prompt, max_length = 150, clean_up_tokenization_spaces = True)
print(outputs[0]['generated_text'])

안녕하세요, 지난 주 독일 아마존 온라인 스토어에서 옵티머스프라임 액션 피규어를 주문했는데, 
받아보니 메가트론 피규어가 와 있더라구요. 디셉티콘은 평생 적인데, 이런 실수는 좀 곤란하네요. 
메가트론 대신 주문한 옵티머스프라임 피규어로 교환해주시면 감사하겠습니다. 
구매 내역 사본도 함께 보냈습니다. 빠른 답변 부탁드릴게요.

고객센터 답변:
아이고 손님 죄송합니다. 당장 주십시요. ᄏ
저희 부부는 옵티머스프라임 구매 후
사용처 : 디셉티콘 / 모바일 어플
설명 : 옵티머스프라임 피규어


## dataset

In [12]:
from datasets import load_dataset

dataset = load_dataset("klue","ynat")
dataset

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [14]:
df = dataset['train']
df[1]

{'guid': 'ynat-v1_train_00001',
 'title': '어버이날 맑다가 흐려져…남부지방 옅은 황사',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008384783',
 'date': '2016.05.08. 오전 5:25'}

## tokenizer

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

In [18]:
encoded_text = tokenizer(text)
encoded_text

{'input_ids': [25906, 8702, 7801, 13704, 12199, 9046, 9865, 25620, 22821, 19684, 23518, 739, 8062, 8608, 15435, 15703, 8152, 46354, 9301, 6944, 10546, 16812, 10713, 9051, 12522, 38306, 47280, 8599, 7428, 9301, 6944, 11813, 10278, 9019, 7208, 7372, 6919, 25856, 10133, 7806, 8608, 8469, 8135, 14999, 739, 9090, 9155, 10165, 9137, 10583, 11732, 29613, 8702, 7098, 25856, 9051, 47826, 8599, 7428, 10276, 16812, 8704, 739, 8062, 8608, 15435, 15703, 8152, 9301, 6944, 10987, 12861, 15084, 7888, 7532, 15940, 26616, 16691, 9051, 35577, 49261, 22772, 7235, 9304, 19083, 16691, 14239, 12817, 7643, 19581, 7281, 7481, 6866, 25856], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
len(encoded_text['input_ids'])

94

In [31]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text['input_ids'])
tokens

['▁안녕',
 '하',
 '세',
 '요,',
 '▁지난',
 '▁주',
 '▁독일',
 '▁아마존',
 '▁온라인',
 '▁스토',
 '어에서',
 '▁',
 '옵',
 '티',
 '머스',
 '프라',
 '임',
 '▁액션',
 '▁피',
 '규',
 '어를',
 '▁주문',
 '했는데,',
 '▁\n',
 '받아',
 '보니',
 '▁메가',
 '트',
 '론',
 '▁피',
 '규',
 '어가',
 '▁와',
 '▁있',
 '더',
 '라',
 '구',
 '요.',
 '▁디',
 '셉',
 '티',
 '콘',
 '은',
 '▁평생',
 '▁',
 '적인',
 '데,',
 '▁이런',
 '▁실',
 '수는',
 '▁좀',
 '▁곤란',
 '하',
 '네',
 '요.',
 '▁\n',
 '메가',
 '트',
 '론',
 '▁대신',
 '▁주문',
 '한',
 '▁',
 '옵',
 '티',
 '머스',
 '프라',
 '임',
 '▁피',
 '규',
 '어로',
 '▁교환',
 '해주',
 '시',
 '면',
 '▁감사',
 '하겠',
 '습니다.',
 '▁\n',
 '구매',
 '▁내역',
 '▁사본',
 '도',
 '▁함께',
 '▁보냈',
 '습니다.',
 '▁빠른',
 '▁답',
 '변',
 '▁부탁',
 '드',
 '릴',
 '게',
 '요.']

In [32]:
print(tokenizer.convert_tokens_to_string(tokens))

안녕하세요, 지난 주 독일 아마존 온라인 스토어에서 옵티머스프라임 액션 피규어를 주문했는데, 
받아보니 메가트론 피규어가 와 있더라구요. 디셉티콘은 평생 적인데, 이런 실수는 좀 곤란하네요. 
메가트론 대신 주문한 옵티머스프라임 피규어로 교환해주시면 감사하겠습니다. 
구매 내역 사본도 함께 보냈습니다. 빠른 답변 부탁드릴게요.


In [33]:
tokenizer.vocab_size

51200

In [34]:
tokenizer.model_max_length

1000000000000000019884624838656

## 모델

샘플 파이토치

In [46]:
import torch
from torch import nn
from torch import optim


x = torch.FloatTensor([
    [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13],[14],[15],
    [16],[17],[18],[19],[20],[21],[22],[23],[24],[25],[26],[27],[28],[29],[30]
])
y = torch.rand(30, 1)*x

model = nn.Linear(1,1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(10000):
  output = model(x)
  cost = criterion(output,y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if (epoch +1 ) % 1000 == 0:
    print(f"Epoch: {epoch+1:4d}, Model: {list(model.parameters())}, Cost: {cost:.3f}")


Epoch: 1000, Model: [Parameter containing:
tensor([[0.5502]], requires_grad=True), Parameter containing:
tensor([-0.6703], requires_grad=True)], Cost: 33.356
Epoch: 2000, Model: [Parameter containing:
tensor([[0.5512]], requires_grad=True), Parameter containing:
tensor([-0.6897], requires_grad=True)], Cost: 33.356
Epoch: 3000, Model: [Parameter containing:
tensor([[0.5518]], requires_grad=True), Parameter containing:
tensor([-0.7018], requires_grad=True)], Cost: 33.356
Epoch: 4000, Model: [Parameter containing:
tensor([[0.5522]], requires_grad=True), Parameter containing:
tensor([-0.7093], requires_grad=True)], Cost: 33.356
Epoch: 5000, Model: [Parameter containing:
tensor([[0.5524]], requires_grad=True), Parameter containing:
tensor([-0.7140], requires_grad=True)], Cost: 33.356
Epoch: 6000, Model: [Parameter containing:
tensor([[0.5525]], requires_grad=True), Parameter containing:
tensor([-0.7169], requires_grad=True)], Cost: 33.356
Epoch: 7000, Model: [Parameter containing:
tensor([[

In [73]:
from transformers import AutoModel
import torch

model_ckpt ="skt/kogpt2-base-v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt)

In [74]:
text = "게임을 시작하지"
inputs = tokenizer(text, return_tensors="pt")


In [75]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
  outputs = model(**inputs)
print(outputs)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.2120,  0.8143,  0.2497,  ..., -1.8134,  1.5043, -2.1539],
         [ 2.0689,  3.9107, -0.5460,  ..., -3.1454,  0.4520, -1.7854],
         [ 0.5654,  0.3654,  1.0527,  ..., -1.1550,  0.3904, -1.9493]]]), past_key_values=((tensor([[[[-7.0791e-01,  6.7755e-01, -5.5809e-01,  ..., -6.4046e-01,
           -6.1706e-01, -3.5399e-01],
          [-2.4302e-01,  3.0404e+00, -1.5970e+00,  ..., -1.8952e+00,
           -1.5896e+00,  3.5532e-02],
          [ 2.0957e-01,  3.3766e+00, -1.6327e+00,  ..., -1.6509e+00,
           -1.3397e-01, -9.6415e-01]],

         [[-2.0817e-01,  4.0657e-01,  4.3409e-01,  ..., -4.5202e-01,
            8.7719e-01,  2.0166e-01],
          [-4.9426e-01,  7.8700e-01,  7.5806e-01,  ..., -8.7090e-01,
           -9.3296e-01,  1.3567e+00],
          [-3.0741e-01,  1.1834e-01, -3.3183e-01,  ...,  4.4986e-01,
           -1.4017e+00,  2.3354e+00]],

         [[-5.9920e-01,  1.2936e+00,  4.8687e-02,  ...,  1.2

In [76]:
outputs.last_hidden_state.size()

torch.Size([1, 3, 768])

## 생성
그리디  
빔  
샘플링  

In [77]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")

output_temp = model.generate(inputs['input_ids'], max_length = 45, do_sample = True,
                             temperature = 2.0, top_k = 0)
tokenizer.decode(output_temp[0])

'게임을 시작하지 않았다.\nNA카드 최종보증했는데, 다음날 불꽃성으로 판매를 항공기 패점 5%를 다음 지지 존재 불가를 언급탄에 소개했다.\n작품명과 줄 저울 돌이 섬 바르셀로나 대통령으로 감내 스트레스주를 형상화한 반발활한 면발을 새겼다.\n목적·성'

In [78]:
output_temp = model.generate(inputs['input_ids'], max_length = 45, do_sample = True,
                             temperature = 0.5, top_k = 0)
tokenizer.decode(output_temp[0])

'게임을 시작하지 않을 경우 이달 20일부터 해당 사업자를 상대로 손해배상 청구 소송을 제기할 계획이다.\nSK텔레콤은 지난해 11월부터 LG유플러스로부터 가입자 유치 경쟁에 나서고 있다.\nLG유플러스는'

In [79]:
output_topk = model.generate(inputs['input_ids'], max_length = 45, do_sample = True,
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

게임을 시작하지 않고 있다.
여덟 달 후 열리는 유럽 축구연맹(UEFA) U-17 월드컵과 2016년 독일에서 개최될 유럽축구연맹(UEFA) U-20 월드컵의 공식 후원 파트너이다.
한국


In [82]:
output_topp = model.generate(inputs['input_ids'], max_length = 45, do_sample = True,
                             top_p=0.90)
print(tokenizer.decode(output_topk[0]))

게임을 시작하지 않고 있다.
여덟 달 후 열리는 유럽 축구연맹(UEFA) U-17 월드컵과 2016년 독일에서 개최될 유럽축구연맹(UEFA) U-20 월드컵의 공식 후원 파트너이다.
한국
